In [2]:

#taken from the several iciar 2018 implementations

import argparse
import numpy as np
import cv2
from scipy import ndimage
from os.path import basename, join, exists
from os import makedirs
#from threaded_generator import threaded_generator
from time import time
import sys
np.random.seed(13)

PATCH_SIZES = [400, 650]
SCALES = [0.5]

DEFAULT_INPUT_DIR = "data/train"
DEFAULT_PREPROCESSED_ROOT = "data/preprocessed/train"

PATCHES_PER_IMAGE = 20
AUGMENTATIONS_PER_IMAGE = 50
COLOR_LO = 0.7
COLOR_HI = 1.3
BATCH_SIZE = 16     # decrease if necessary

NUM_CACHED = 160


In [5]:
def normalize_staining(img):
   
    Io = 240
    beta = 0.15
    alpha = 1
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
    maxCRef = np.array([1.9705, 1.0308])

    h, w, c = img.shape
    img = img.reshape(h * w, c)
    OD = -np.log((img.astype("uint16") + 1) / Io)
    ODhat = OD[(OD >= beta).all(axis=1)]
    W, V = np.linalg.eig(np.cov(ODhat, rowvar=False))

    Vec = -V.T[:2][::-1].T  
    That = np.dot(ODhat, Vec)
    phi = np.arctan2(That[:, 1], That[:, 0])
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100 - alpha)
    vMin = np.dot(Vec, np.array([np.cos(minPhi), np.sin(minPhi)]))
    vMax = np.dot(Vec, np.array([np.cos(maxPhi), np.sin(maxPhi)]))
    if vMin[0] > vMax[0]:
        HE = np.array([vMin, vMax])
    else:
        HE = np.array([vMax, vMin])

    HE = HE.T
    Y = OD.reshape(h * w, c).T

    C = np.linalg.lstsq(HE, Y)
    maxC = np.percentile(C[0], 99, axis=1)

    C = C[0] / maxC[:, None]
    C = C * maxCRef[:, None]
    Inorm = Io * np.exp(-np.dot(HERef, C))
    Inorm = Inorm.T.reshape(h, w, c).clip(0, 255).astype("uint8")

    return Inorm


def hematoxylin_eosin_aug(img, low=0.7, high=1.3, seed=None):
    D = np.array([[1.88, -0.07, -0.60],
                  [-1.02, 1.13, -0.48],
                  [-0.55, -0.13, 1.57]])
    M = np.array([[0.65, 0.70, 0.29],
                  [0.07, 0.99, 0.11],
                  [0.27, 0.57, 0.78]])
    Io = 240

    h, w, c = img.shape
    OD = -np.log10((img.astype("uint16") + 1) / Io)
    C = np.dot(D, OD.reshape(h * w, c).T).T
    r = np.ones(3)
    r[:2] = np.random.RandomState(seed).uniform(low=low, high=high, size=2)
    img_aug = np.dot(C, M) * r

    img_aug = Io * np.exp(-img_aug * np.log(10)) - 1
    img_aug = img_aug.reshape(h, w, c).clip(0, 255).astype("uint8")
    return img_aug

def zoom_aug(img, zoom_var, seed=None):
    scale = np.random.RandomState(seed).uniform(low=1 / zoom_var, high=zoom_var)
    resized_img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    return resized_img

def get_crops(img, size, n, seed=None):
    h, w, c = img.shape
    assert all([size < h, size < w])
    crops = []
    for _ in range(n):
        top = np.random.randint(low=0, high=h - size + 1)
        left = np.random.randint(low=0, high=w - size + 1)
        crop = img[top: top + size, left: left + size].copy()
        crop = np.rot90(crop, np.random.randint(low=0, high=4))
        if np.random.random() > 0.5:
            crop = np.flipud(crop)
        if np.random.random() > 0.5:
            crop = np.fliplr(crop)
        crops.append(crop)

    crops = np.stack(crops)
    assert crops.shape == (n, size, size, c)
    return crops


def get_crops_free(img, size, n, seed=None):
    h, w, c = img.shape
    assert all([size < h, size < w])
    d = int(np.ceil(size / np.sqrt(2)))
    crops = []
    for _ in range(n):
        center_y = np.random.randint(low=0, high=h - size + 1) + size // 2
        center_x = np.random.randint(low=0, high=w - size + 1) + size // 2
        m = min(center_y, center_x, h - center_y, w - center_x)
        if m < d:
            max_angle = np.pi / 4 - np.arccos(m / d)
            top = center_y - m
            left = center_x - m
            precrop = img[top: top + 2 * m, left: left + 2 * m]
        else:
            max_angle = np.pi / 4
            top = center_y - d
            left = center_x - d
            precrop = img[top: top + 2 * d, left: left + 2 * d]

        precrop = np.rot90(precrop, np.random.randint(low=0, high=4))
        angle = np.random.uniform(low=-max_angle, high=max_angle)
        precrop = ndimage.rotate(precrop, angle * 180 / np.pi, reshape=False)

        precrop_h, precrop_w, _ = precrop.shape
        top = (precrop_h - size) // 2
        left = (precrop_w - size) // 2
        crop = precrop[top: top + size, left: left + size]

        if np.random.random() > 0.5:
            crop = np.flipud(crop)
        if np.random.random() > 0.5:
            crop = np.fliplr(crop)
        crops.append(crop)

    crops = np.stack(crops)
    assert crops.shape == (n, size, size, c)
    return crops


def norm_pool(features, p=3):
    return np.power(np.power(features, p).mean(axis=0), 1/p)


def encode(crops, model):
    features = model.predict(crops)
    pooled_features = norm_pool(features)
    return pooled_features


def process_image(image_file):
    img = cv2.imread(image_file)
    if SCALE != 1:
        img = cv2.resize(img, None, fx=SCALE, fy=SCALE, interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_norm = normalize_staining(img)

    for _ in range(AUGMENTATIONS_PER_IMAGE):
        img_aug = hematoxylin_eosin_aug(img_norm, low=COLOR_LO, high=COLOR_HI)
        single_image_crops = get_crops(img_aug, PATCH_SZ, PATCHES_PER_IMAGE)
        yield single_image_crops


def crops_gen(file_list):
    for i, (image_file, output_file) in enumerate(file_list):
        print("Crops generator:", i + 1)
        for crops in process_image(image_file):
            yield crops, output_file


def features_gen(crops_and_output_file, model):
    ts = time()
    current_file = None
    pooled_features = []
    i = 0
    for j, (crops, output_file) in enumerate(crops_and_output_file):
        if current_file is None:
            current_file = output_file
        features = encode(crops, model)
        if output_file == current_file:
            pooled_features.append(features)
        else:
            np.save(current_file, np.stack(pooled_features))
            pooled_features = [features]
            current_file = output_file
            average_time = int((time() - ts) / (i + 1))
            print("Feature generator: {}, {} sec/image.".format(i + 1, average_time))
            i += 1
    if len(pooled_features) > 0:
        np.save(current_file, np.stack(pooled_features))





In [15]:
DATASET_PATH = 'E:\Projects\HolisticClassification\TempDataset' # the dataset file or root folder path.
TEST_DATASET_PATH = 'E:\Projects\HolisticClassification\TempDataset\Test'

BATCH_SIZE = 4
MODE = 'folder' # or 'file', if you choose a plain text file (see above).
from PIL import Image
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time 

import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
#define 
def read_images_convert_to_jpeg(dataset_path = DATASET_PATH, mode = MODE, batch_size = BATCH_SIZE):
    imagepaths, labels = list(), list()
    if mode == 'folder':
        label = 0
        classes = [f.path for f in os.scandir(dataset_path) if f.is_dir() ] 
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            walk = os.walk(c_dir).__next__()
            for sample in walk[2]:
                if sample.endswith('.BMP') or sample.endswith('.bmp'):
                    imagepaths.append(os.path.join(c_dir, sample))
                    labels.append(label)
            label += 1
    else:
        raise Exception("Unknown mode.")
    for image_loc in imagepaths:
        new_img = Image.open(image_loc)
        save_loc = os.path.splitext(image_loc)[0] + ".jpeg"
        print(save_loc)
        new_img.save( save_loc, 'jpeg')
        

In [16]:
read_images_convert_to_jpeg()

E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_10.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_11.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_12.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_13.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_14.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_15.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_16.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_17.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_18.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_19.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_2.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_20.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0\Compressed_3.jpeg
E:\Projects\HolisticClassification\TempDataset\Class0

E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_32.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_33.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_34.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_35.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_36.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_37.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_38.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_39.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_40.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_41.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_42.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_43.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Compressed_44.jpeg
E:\Projects\HolisticClassification\TempDataset\Clas

E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_139.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_140.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_141.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_142.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_143.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_144.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_145.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_146.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_147.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_148.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_149.jpeg
E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_1

E:\Projects\HolisticClassification\TempDataset\Class1\Rotated180Compressed_99.jpeg
